In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import sys
# sys.path.insert(0, "mlstm_simple_torch")

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers

/nfs-gpu/xlstm/miniforge3/envs/xlstm_eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import xLSTMModel, xLSTMForCausalLM, xLSTMConfig, xLSTMPreTrainedModel
from transformers.models.xlstm.modeling_xlstm import xLSTMCache

2024-11-24 18:05:19.300418: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 18:05:19.362134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 18:05:19.362169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 18:05:19.364028: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 18:05:19.374817: I tensorflow/core/platform/cpu_feature_guar

In [5]:
# ckpt_path = "/nfs-gpu/xlstm/logs/converted_model_checkpoints/dclm_mLSTMv1_7B_ctx8192_2024-11-19T11:12:49_505000"
# model = xLSTMForCausalLM.from_pretrained(ckpt_path, device_map='auto')

model = AutoModelForCausalLM.from_pretrained("/nfs-gpu/xlstm/logs/outputs_poeppel/tmpckpt/checkpoint_test/testmodel10", device_map='auto')

Loading checkpoint shards: 100%|██████████| 6/6 [00:08<00:00,  1.37s/it]


In [6]:
model.backbone.blocks[0].config.forward_backend_name

'chunkwise--triton_limit_chunk'

In [7]:
for param_name, _ in model.named_parameters():
    # print(param_name)
    pass

In [ ]:
model.backbone.embeddings.weight

Parameter containing:
tensor([[-4.1347e-03,  6.2107e-02,  1.3004e-01,  ..., -3.9270e-02,
         -8.5980e-02,  5.1134e-01],
        [-2.7021e-03,  8.1594e-03, -1.2019e-02,  ..., -5.8456e-03,
         -6.2147e-03,  4.4870e-03],
        [-1.4784e-02,  9.8308e-03,  4.3885e-03,  ...,  1.2567e-02,
          3.5710e-03, -1.4621e-02],
        ...,
        [-1.3915e-03, -6.5551e-03, -8.1581e-04,  ..., -1.2919e-03,
          6.9401e-03,  7.1876e-03],
        [-2.6722e-03, -6.9579e-03, -8.0867e-03,  ..., -7.8452e-03,
         -9.5306e-03,  4.7927e-03],
        [ 5.0381e-03,  1.3760e-03, -3.2230e-03,  ..., -7.5782e-03,
         -9.6426e-03,  2.7401e-04]], device='cuda:0', requires_grad=True)

In [9]:
import torch

tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')
# tokens = model.tokenize("Hello My name is Computer.")

input_ids = tokenizer("The play, celebrated for its wit and repartee, parodies contemporary dramatic norms and comically satirises late-Victorian manners. The triumphant opening night was", return_tensors="pt")

In [10]:
model.config.to_mlstm_block_config().forward_backend_name

'chunkwise--triton_limit_chunk'

In [11]:
input_ids

{'input_ids': tensor([[  510,  1132,    13, 19651,   323,   697, 19311,   285,  1234,   435,
          1796,    13,  1061,  4550, 13399, 14138, 22429,   285,   389,  1037,
          2206,   343,  3013,  3563,    14, 35419,   757, 34323,    15,   380,
         19725, 13645,  5909,  2360,   369]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [14]:
out = model.generate(input_ids['input_ids'].to(device="cuda"), use_cache=True, max_new_tokens=40)

In [15]:
tokenizer.decode(out[0])

'The play, celebrated for its wit and repartee, parodies contemporary dramatic norms and comically satirises late-Victorian manners. The triumphant opening night was its first night. The triumphant opening night was the first of its first night. The late-Victorian late-Victorian. The late-Victorian late-Victorian. The late-'

In [14]:
# model.save_pretrained('/nfs-gpu/xlstm/logs/outputs_poeppel/tmpckpt/checkpoint_test/testmodel10')